# 1편: 이미지의 디지털 표현과 신경망 첫걸음

이 노트북에서는 다음을 학습합니다:
- 이미지가 컴퓨터에서 어떻게 숫자 배열로 표현되는지
- RGB 채널의 의미와 분리 방법
- PyTorch 텐서와 Linear 레이어 기초
- 학습의 핵심 사이클: Forward → Loss → Backward → Step

## 1. 이미지는 숫자다

컴퓨터가 보는 이미지는 픽셀 값으로 이루어진 3차원 배열입니다.
- **height**: 세로 픽셀 수
- **width**: 가로 픽셀 수
- **channels**: 색상 채널 수 (RGB = 3)

In [ ]:
from PIL import Image
import numpy as np
import copy
import urllib.request
import os

# 샘플 이미지 다운로드
img_url = "https://upload.wikimedia.org/wikipedia/commons/thumb/4/47/PNG_transparency_demonstration_1.png/280px-PNG_transparency_demonstration_1.png"
img_path = "sample_image.png"
if not os.path.exists(img_path):
    urllib.request.urlretrieve(img_url, img_path)

img = Image.open(img_path).convert("RGB")
img

In [ ]:
np_img = np.array(img)
print(f"이미지 shape: {np_img.shape}")
print(f"  → 높이={np_img.shape[0]}px, 너비={np_img.shape[1]}px, 채널={np_img.shape[2]}(RGB)")
print(f"\n픽셀 값 범위: {np_img.min()} ~ {np_img.max()}")
print(f"\n좌측 상단 5x5 영역의 Red 채널 값:\n{np_img[:5, :5, 0]}")

### 이미지 크롭(Crop)

numpy 슬라이싱으로 이미지의 특정 영역을 잘라낼 수 있습니다.

In [ ]:
# 이미지 중앙 영역을 크롭
h, w = np_img.shape[:2]
crop_h, crop_w = 100, 150
start_h, start_w = h // 2 - crop_h // 2, w // 2 - crop_w // 2
cropped = np_img[start_h:start_h+crop_h, start_w:start_w+crop_w]

print(f"크롭 전: {np_img.shape} → 크롭 후: {cropped.shape}")
Image.fromarray(cropped)

### RGB 채널 분리

이미지는 Red, Green, Blue 3개 채널로 구성됩니다. 각 채널만 남기면 해당 색상 성분을 확인할 수 있습니다.

In [ ]:
import matplotlib.pyplot as plt

fig, axes = plt.subplots(1, 4, figsize=(16, 4))

# 원본
axes[0].imshow(cropped)
axes[0].set_title("Original")

# 각 채널 분리
channel_names = ["Red", "Green", "Blue"]
for i, name in enumerate(channel_names):
    channel_img = np.zeros_like(cropped)
    channel_img[:, :, i] = cropped[:, :, i]
    axes[i + 1].imshow(channel_img)
    axes[i + 1].set_title(f"{name} Channel")

for ax in axes:
    ax.axis("off")
plt.tight_layout()
plt.show()

In [ ]:
# 그레이스케일 변환
grey = np.mean(cropped, axis=2).astype(np.uint8)
print(f"그레이스케일 shape: {grey.shape} (채널 차원이 사라짐)")

fig, axes = plt.subplots(1, 2, figsize=(8, 4))
axes[0].imshow(cropped)
axes[0].set_title("Color")
axes[1].imshow(grey, cmap="gray")
axes[1].set_title("Grayscale")
for ax in axes:
    ax.axis("off")
plt.tight_layout()
plt.show()

## 2. PyTorch 텐서 소개

PyTorch는 GPU 가속을 지원하는 텐서 연산 라이브러리입니다.
numpy 배열과 유사하지만, **자동 미분(autograd)** 기능이 핵심입니다.

In [ ]:
import torch

# numpy 배열 → PyTorch 텐서
np_arr = np.array([1.0, 2.0, 3.0])
tensor = torch.from_numpy(np_arr)
print(f"numpy:  {np_arr}, dtype={np_arr.dtype}")
print(f"tensor: {tensor}, dtype={tensor.dtype}")

# 디바이스 설정
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"\n사용 디바이스: {device}")

# 텐서 연산
a = torch.tensor([1.0, 2.0, 3.0])
b = torch.tensor([4.0, 5.0, 6.0])
print(f"\n덧셈: {a + b}")
print(f"행렬곱: {torch.dot(a, b)}")

## 3. 신경망의 가장 작은 단위: Linear 레이어

Linear 레이어는 다음 연산을 수행합니다:

$$y = xW^T + b$$

- **x**: 입력 (input_features 차원)
- **W**: 가중치 행렬 (output_features × input_features)
- **b**: 편향 벡터 (output_features 차원)
- **y**: 출력 (output_features 차원)

In [ ]:
import torch.nn as nn

# 4차원 입력 → 2차원 출력 Linear 레이어
linear = nn.Linear(4, 2, bias=True).to(device)

print(f"가중치(W) shape: {linear.weight.shape}")
print(f"편향(b) shape:   {linear.bias.shape}")
print(f"\n초기 가중치:\n{linear.weight.data}")
print(f"초기 편향: {linear.bias.data}")

## 4. 학습의 핵심 사이클

딥러닝 학습은 4단계의 반복입니다:

1. **Forward (순전파)**: 입력을 모델에 통과시켜 예측값을 계산
2. **Loss (손실 계산)**: 예측값과 정답의 차이를 측정
3. **Backward (역전파)**: 손실에 대한 각 가중치의 기울기를 계산
4. **Step (가중치 업데이트)**: 기울기 방향으로 가중치를 조정

In [ ]:
# 간단한 분류 문제 설정
inputs = np.array([2.0, 4.0, 5.0, 6.0])
target_class = 0  # 클래스 0으로 분류되어야 하는 입력

X = torch.Tensor(inputs).view(1, -1).to(device)   # (1, 4) 형태
Y = torch.tensor([target_class]).to(device)        # 정답 레이블

# 손실 함수와 옵티마이저 설정
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.SGD(linear.parameters(), lr=0.1)

print(f"입력 X shape: {X.shape}")
print(f"정답 Y: {Y}")

In [ ]:
# === 학습 1 사이클 상세 관찰 ===

# 1단계: Forward (순전파)
optimizer.zero_grad()  # 기울기 초기화 (누적 방지)
prediction = linear(X)
print(f"[Forward] 예측값: {prediction.data}")
print(f"  → 클래스 0 점수: {prediction.data[0][0]:.4f}, 클래스 1 점수: {prediction.data[0][1]:.4f}")

# 2단계: Loss (손실 계산)
loss = criterion(prediction, Y)
print(f"\n[Loss] 손실값: {loss.item():.4f}")

# 3단계: Backward (역전파) - 기울기 계산
print(f"\n[Backward 전] weight.grad: {linear.weight.grad}")
loss.backward()
print(f"[Backward 후] weight.grad:\n{linear.weight.grad}")

# 4단계: Step (가중치 업데이트)
weight_before = linear.weight.data.clone()
optimizer.step()
weight_after = linear.weight.data.clone()

print(f"\n[Step] 가중치 변화:")
print(f"  Before: {weight_before[0][:2].tolist()}...")
print(f"  After:  {weight_after[0][:2].tolist()}...")
print(f"  차이:   {(weight_after - weight_before)[0][:2].tolist()}...")

## 5. 반복 학습으로 손실 줄이기

위의 4단계를 여러 번 반복하면, 가중치가 점차 최적값에 수렴하며 손실이 줄어듭니다.

In [ ]:
# 20 에포크 학습
losses = []
for epoch in range(20):
    optimizer.zero_grad()
    prediction = linear(X)
    loss = criterion(prediction, Y)
    loss.backward()
    optimizer.step()
    losses.append(loss.item())
    if (epoch + 1) % 5 == 0:
        print(f"Epoch {epoch+1:02d} | loss = {loss.item():.6f}")

# 학습 과정 시각화
plt.figure(figsize=(8, 4))
plt.plot(range(1, 21), losses, marker="o", markersize=4)
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Loss Curve")
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"\n최종 가중치:\n{linear.weight.data}")

## 정리

| 개념 | 설명 |
|---|---|
| 이미지 | 숫자로 이루어진 3차원 배열 (H × W × C) |
| 텐서 | PyTorch의 기본 데이터 구조 (GPU 연산 + 자동 미분 지원) |
| Linear 레이어 | y = xW^T + b 연산을 수행하는 신경망의 기본 단위 |
| 학습 사이클 | Forward → Loss → Backward → Step 의 반복 |

**다음 노트북에서는** 실제 이미지 데이터셋(FashionMNIST)을 사용하여 완전연결 신경망을 학습해봅니다.